In [2]:
import torchvision
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure
from sklearn.model_selection import train_test_split

import timeit
from sklearn.metrics import accuracy_score
from snorkel.labeling import labeling_function
from snorkel.labeling import LFApplier
from snorkel.labeling import LFAnalysis

ModuleNotFoundError: No module named 'torchvision'

In [3]:
!pip install snorkel

  Using cached snorkel-0.9.7-py3-none-any.whl (145 kB)
  Using cached networkx-2.3-py2.py3-none-any.whl
  Using cached scikit_learn-0.24.2-cp38-cp38-win_amd64.whl (6.9 MB)
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
  Using cached munkres-1.1.4-py2.py3-none-any.whl (7.0 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached Markdown-3.3.4-py3-none-any.whl (97 kB)
  Using cached Werkzeug-2.0.1-py3-none-any.whl (288 kB)
  Using cached absl_py-0.13.0-py3-none-any.whl (132 kB)
  Using cached typing_extensions-3.10.0.0-py3-none-any.whl (26 kB)


In [4]:
np.random.seed(0)

In [5]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [6]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.FashionMNIST(root = 'FashionMNIST/processed/training.pt', train=True, download=True, transform=transform)
train_data_loader = torch.utils.data.DataLoader(train_data,batch_size=train_data.__len__(),shuffle=True)
train_images, train_labels = next(iter(train_data_loader))

In [7]:
test_data = torchvision.datasets.FashionMNIST(root = 'FashionMNIST/processed/testing.pt', train=True, download=True, transform=transform)
test_data_loader = torch.utils.data.DataLoader(test_data,batch_size=test_data.__len__(),shuffle=True)
test_images, test_labels = next(iter(test_data_loader))

In [8]:
def baseline_knn_processing(image):
    return np.ravel(image.cpu().detach().numpy())

In [9]:
def label_processing(label):
    return label.cpu().detach().numpy().item()

In [23]:
X_train = list(map(baseline_knn_processing, train_images))
y_train = list(map(label_processing, train_labels))
X_test = list(map(baseline_knn_processing, test_images))
y_test = list(map(label_processing, test_labels))

In [24]:
neigh_1 = KNeighborsClassifier(n_neighbors=12, p = 1)
neigh_1.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=12, p=1)

In [25]:
neigh_2 = KNeighborsClassifier(n_neighbors=12, p = 2)
neigh_2.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=12)

In [26]:
num_sample = 10
chosen_index = np.random.choice(len(X_test), num_sample)
X_test_sample = np.array(X_test)[chosen_index]
y_test_sample = np.array(y_test)[chosen_index]

In [27]:
##y_pred = neigh.predict(X_test_sample)
#accuracy_score(y_test_sample, y_pred)

In [29]:
@labeling_function()
def knn_1(x):
    x = np.expand_dims(x, axis=0)
    return neigh_1.predict(x)[0]

In [30]:
@labeling_function()
def knn_2(x):
    x = np.expand_dims(x, axis=0)
    return neigh_2.predict(x)[0]

In [31]:
lfs = [knn_1, knn_2]
applier = LFApplier(lfs)
L_train = applier.apply(X_test_sample)
LFAnalysis(L_train, lfs).lf_summary()

10it [00:05,  1.77it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
knn_1,0,"[0, 1, 2, 3, 4, 5, 6]",1.0,1.0,0.1
knn_2,1,"[0, 1, 2, 3, 4, 5, 6]",1.0,1.0,0.1


In [2]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=10, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

C:\Users\Nutchapold\Anaconda3\envs\growsquares_environment\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nutchapold\Anaconda3\envs\growsquares_environment\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Nutchapold\Anaconda3\envs\growsquares_environment\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


ImportError: cannot import name 'gcd' from 'fractions' (C:\Users\Nutchapold\Anaconda3\envs\growsquares_environment\lib\fractions.py)

In [3]:
from platform import python_version

print(python_version())

3.9.2


In [43]:
def baseline_knn_processing(image):
    image = image.cpu().detach().numpy()
    image = np.squeeze(image)
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(4, 4),
                        cells_per_block=(1, 1), visualize=True)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return np.ravel(hog_image_rescaled)

In [44]:
X = list(map(baseline_knn_processing, images[:1000]))

In [45]:
y = list(map(label_processing, labels[:1000]))

In [46]:
neigh = KNeighborsClassifier(n_neighbors=12)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [48]:
X_train[0].shape

(784,)

In [49]:
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=12)

In [50]:
y_pred = neigh.predict(X_test)

In [51]:
np.sum(y_pred == np.ravel(y_test)) / len(y_test)

0.706060606060606